# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [12]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/Benchmarks/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data_set = 'ionosphere'  #@param {type: "string"}

#cleaned data without non-attack values
X = pd.read_csv(root_path+data_set+'.csv' , usecols=[i for i in range(34)])
Y = pd.read_csv(root_path+data_set+'.csv', usecols=[34])

# Feature Engineering

In [14]:
# cluster and score
from sklearn.cluster import Birch
from sklearn.metrics.cluster import fowlkes_mallows_score as scorer
score = []
for i in range(len(X.columns)): # loop number of features
  K = Birch(n_clusters=len(Y[Y.columns[0]].unique())-1)
  print(X.iloc[:, [i]].values.shape, i, "begin")
  pred = K.fit_predict(X.iloc[:, [i]].values)
  print(X.iloc[:, [i]].values.shape, i, "complete")
  s = scorer(Y[Y.columns[0]].values,pred)
  score.append(s)
  
  
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 



    
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]
  
  

# imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(n_estimators=10,random_state=0, n_jobs=-1, max_depth=5)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 
  

  
# best score calcuation
index = accuracy.index(max(accuracy))

X = X2.iloc[:,0:len(X.columns)-index]

(350, 1) 0 begin


ValueError: ignored

# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

In [0]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

In [0]:

x = list(zip(X.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X.columns)):
    print(x[i])
    if (i==len(X.columns)-1):
      print("features",i+1)

In [0]:
print(clf.score(x_train, y_train), "train")

In [0]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))